In [223]:
import xmltodict
import xml.etree.ElementTree as et
import math

In [110]:
with open("xmls/Whole_leg_sim_Standalone.asim",'r') as f:
    dicto = xmltodict.parse(f.read())
rat = dicto['Simulation']['Environment']["Organisms"]["Organism"]

In [386]:
def process_rb(rb):
    new_rb = {}
    keys = ["Name","ID","Mass","Density","Position","Rotation","Type",
            "IsCollisonObject","IsContactSensor"
            "StimulusTension","Length Tension","MaximumTension","Kse","Kpe","B","Attachments"]
    for key,value in rb.items():
        if key in keys:
            new_rb[key] = process_item(key,value)
    
    if "ChildBodies" in rb.keys():
        new_rb["ChildBodies"] = [process_rb(child) for child in rb["ChildBodies"]["RigidBody"]]
    if "Width" in rb.keys(): #if width exists, assume the rest do...
        new_rb["size"] = "{} {} {}".format(float(rb["Width"])*50,float(rb["Length"])*50,float(rb["Height"])*50)
    if "Joint" in rb.keys():
        if isinstance(rb["Joint"],list):
            new_rb["Joints"] = [process_joint(child) for child in rb["Joint"]]
        else:
            new_rb["Joint"] = process_joint(rb["Joint"])
    
    return new_rb

def process_joint(joint):
    new_jnt = {}
    keys = ["Name","Position","Rotation","Type","Size",
            "MaxForce","MaxVelocity","EnableMotor","MotorType","ServoGain"]
    relaxations = []
    for key,value in joint.items():
        if key in keys:
            new_jnt[key] = process_item(key,value)
        if key in ["LowerLimit","UpperLimit"]:
            new_jnt[key] = {}
            new_jnt[key]["LimitPos"] = float(value["LimitPos"])
            new_jnt[key]["Damping"] = float(value["Damping"])
            new_jnt[key]["Restitution"] = float(value["Restitution"])
            new_jnt[key]["Stiffness"] = float(value["Stiffness"]) 
        if key == "Friction":
            if value["Enabled"] == True:
                new_jnt["Friction"] = {}
                new_jnt["Friction"]["Type"] = float(value["Type"])
                new_jnt["Friction"]["Coefficient"] = float(value["Coefficient"])
                new_jnt["Friction"]["MaxForce"] = float(value["MaxForce"])
                new_jnt["Friction"]["Loss"] = float(value["Loss"])
                
            
    return new_jnt

def process_item(key,param):
    if key =="Position":
        return "{} {} {}".format(float(param["@x"])*100,float(param["@z"])*100,float(param["@y"])*100)
    elif key =="Rotation":
        return "{} {} {}".format(
            float(param["@x"])*180 / math.pi,
            float(param["@z"])*180 / math.pi,
            float(param["@y"])*180 / math.pi)
    else:
        try:
            float(param)
            return float(param)
        except:
            return param

In [387]:
dic = process_rb(rat["RigidBody"])
#for i in rat["RigidBody"]["ChildBodies"]["RigidBody"]:
#    if "Joint" in i.keys():
#        print(i["Joint"])


In [388]:
#[body["Name"] for body in dic["ChildBodies"][5]["ChildBodies"]]
item = dic
print(item["Name"])
print(item["size"])
#print(item["Rotation"])
#print(item["Attachments"])

Pelvis
0.8500000000000001 1.55 1.0


In [389]:
def xmlify_body(dic,attach_ids={},root=0):
    #print(dic["Name"])
    #print(dic["Type"])
    muscles = []
    body = et.Element("body",attrib={
        "name":dic["Name"],
        "pos":dic["Position"],
        "euler":dic["Rotation"]
    })
    body.append(et.Element("geom",attrib={
        "type": "box",
        "size": dic["size"]
    }))
    if "ChildBodies" in dic.keys():
        for value in dic["ChildBodies"]:
            if value["Type"] == "Attachment":
                body.append(et.Element("site",attrib={
                    "name":value["Name"],
                    "pos":value["Position"]
                }))
                attach_ids[value["ID"]] = value["Name"]
            if value["Type"].__contains__("Muscle"):
                muscles.append(value)
            if value["Type"] == "Box":
                child, muscles, _ = xmlify_body(value,attach_ids=attach_ids,root=1)
                body.append(child)
                muscles.append(muscles)
    if "Joint" in dic.keys():
            joints = []
            if isinstance(dic["Joint"],list):
                joints = dic["Joint"]
            else:
                joints.append(dic["Joint"])
            for value in joints:
                body.append(et.Element("joint",attrib={
                    "name":value["Name"],
                    "pos":value["Position"],
                    "axis":value["Rotation"]
                }))

    if root == 0:
        pass
        #for muscle in muscles:
    return body, muscles, attach_ids

In [390]:
root = et.Element("mujoco")
root.append(et.Element("compiler",attrib={"coordinate":"local","angle":"degree"}))

default = et.SubElement(root,"default")
default.append(et.Element("geom",attrib={"rgba":".8 .4 .6 1"}))

asset = et.SubElement(root,"asset")
asset.append(et.Element("texture",attrib={"type":"skybox",
                                          "builtin":"gradient",
                                          "rgb1":"1 1 1","rgb2":".6 .8 1",
                                          "width":"256",
                                          "height":"256"
                                         }))
world_body = et.SubElement(root,"worldbody")
world_body.append(et.Element("geom",attrib={"name":"floor",
                                            "pos":"0 0 -5",
                                            "size":"10 10 0.125",
                                            "type":"plane",
                                            "condim":"3",
                                            "rgba":"1 1 1 1"
                                           }))
world_body.append(et.Element("light",attrib={"pos":"0 5 5",
                                            "dir":"0 -1 -1",
                                            "diffuse":"1 1 1"
}))
body,muscles,attach_ids = xmlify_body(dic)
world_body.append(body)
with open("xmls/NewRatBastard.xml","wb") as fp: 
    et.indent(root)
    tree = et.ElementTree(root)
    tree.write(fp)

In [385]:
import json
with open('outfile.json','w') as fp:
    json.dump(dic,fp)